# ECB Data Academy - Week 3 - Supervised Learning
[Krisolis](http://www.krisolis.ie)

## Evaluating Predictive Models

This notebook demonstrates how predictive models are evaluated in sklearn.

### Package Imports

To build predictive models in Python we use a set of libraries that are imported here. In particular **pandas** and **sklearn** are particularly important.

In [ ]:
# Saving Python ojects
import os
import pickle

# General data handling
import pandas as pd
import numpy as np

# Drawing plots
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#import pandas_profiling

# Machine learning
import sklearn
import sklearn.impute
import sklearn.model_selection
import sklearn.metrics
import sklearn.tree
import sklearn.svm
import sklearn.ensemble
import sklearn.linear_model
import sklearn.neighbors

### Load & Partition Data

#### Load Data

To support data exploration and manipulation it is easiest o load datasets as **pandas DataFrames**. 

In [ ]:
dataset = pd.read_csv('../data/ACME_ABT.csv')
display(dataset.head(10))

#### Explore the Dataset

Examine the distribution of the two classes

In [ ]:
dataset["churn"].value_counts()

Generate a suite of summary statistics for the numeric and categorical features in the data, and count missing values. 

In [ ]:
# Print descriptive statsitcs for each column
print("Summary Stats")
if dataset.select_dtypes(include=[np.number]).shape[1] > 0: 
    display(dataset.describe(include="number").transpose())
if dataset.select_dtypes(include=[object]).shape[1] > 0: 
    display(dataset.describe(include="object").transpose())

# Check for presence of missing values
print("Missing Values")
print(dataset.isnull().sum())

A **ProfileReport** from **pandas_profiling** gives a very useful summary of the dataset and highliughts potential issues. 

In [ ]:
#pandas_profiling.ProfileReport(dataset, 
                               #minimal = True)

In [ ]:
# A bug in pandas_profiler means plots don;t appear after calling it. This re-import of matplotlib fixes the bug.
import matplotlib.pyplot as plt
%matplotlib inline

#### Prepare Data for Modelling

We select features, imp-iuts missing values, replace spurious values and change categorical features to numeric. 

In [ ]:
X = dataset[['age',
 'income',
 'numHandsets',  
 'handsetAge',
 'smartPhone',
 'currentHandsetPrice',
 'avgBill',
 'avgOverBundleMins',
 'avgRoamCalls',
 'callMinutesChangePct',
 'callMinutesChangePct',
 'billAmountChangePct',
 'billAmountChangePct',
 'avgReceivedMins',
 'avgOutCalls',
 'avgInCalls',
 'peakOffPeakRatio',
 'peakOffPeakRatioChangePct',
 'avgDroppedCalls',
 'avgDroppedCalls',
 'lifeTime',
 'newFrequentNumbers',
 'regionType',
 'marriageStatus',
 'creditRating']]
y = dataset["churn"]

In [ ]:
X.loc[X['regionType'] == 's','regionType'] = "suburban"
X.loc[X['regionType'] == 't','regionType'] = "town"
X.loc[X['regionType'] == 'r','regionType'] = "rural"

In [ ]:
regionType_imputer = sklearn.impute.SimpleImputer(strategy="most_frequent")
regionType_imputer.fit(X['regionType'].values.reshape(-1, 1))
X['regionType'] = regionType_imputer.transform(X['regionType'].values.reshape(-1, 1))

age_imputer = sklearn.impute.SimpleImputer(missing_values = 0, strategy="mean")
age_imputer.fit(X['age'].values.reshape(-1, 1))
X['age'] = age_imputer.transform(dataset['age'].values.reshape(-1, 1))

In [ ]:
creditRating_oe = sklearn.preprocessing.OrdinalEncoder()
creditRating_oe.fit(X['creditRating'].values.reshape(-1, 1))
X['creditRating'] = creditRating_oe.transform(X['creditRating'].values.reshape(-1, 1))

In [ ]:
X = pd.get_dummies(X)

#### Examine Transformed Data

Examine the transformed dataset before modelling.

In [ ]:
print(X.shape)
display(X.head())

In [ ]:
X.columns

In [ ]:
pandas_profiling.ProfileReport(X, 
                               minimal = True)

In [ ]:
# A bug in pandas_profiler means plots don;t appear after calling it. This re-import of matplotlib fixes the bug.
import matplotlib.pyplot as plt
%matplotlib inline

#### Partition Data

Split the data into a **training set**, a **validation set**, and a **test set**.

In [ ]:
X_train_plus_valid, X_test, y_train_plus_valid, y_test \
    = sklearn.model_selection.train_test_split(X, y, random_state=0,
                                               train_size = 0.7, 
                                               stratify = y)

X_train, X_valid, y_train, y_valid \
    = sklearn.model_selection.train_test_split(X_train_plus_valid, 
                                               y_train_plus_valid, 
                                               random_state=0, 
                                               train_size = 0.5/0.7,
                                               stratify=y_train_plus_valid)

Examine the partitions created. 

In [ ]:
print(X_train.shape)
display(X_train.head())

In [ ]:
print(X_valid.shape)
display(X_valid.head())

In [ ]:
print(y_train.shape)
display(y_train.head())

### Model Training

We train a simple decision tree model. 

In [ ]:
my_tree = sklearn.tree.DecisionTreeClassifier(criterion="entropy", 
                                              max_depth=5)
my_tree = my_tree.fit(X_train,y_train)

Visualise the decision tree so we can see what it is doing!

In [ ]:
fig = plt.figure(figsize=(10,10))
_ = sklearn.tree.plot_tree(my_tree, 
                feature_names=X_train.columns,
                  # class_names=iris.target_names,
                   filled=True)

### Standard Evaluation

Our standard evalaution block for classification problems generates the classification report and a confusion matrix. 

In [ ]:
# Make a set of predictions for the training data
y_pred = my_tree.predict(X_train)

# Print performance details
print(sklearn.metrics.classification_report(y_train, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_train, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
# Make a set of predictions for the training data
y_pred = my_tree.predict(X_valid)

# Print performance details
print(sklearn.metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_valid, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Using **sklearn** we can calculate a whole raft of performance measures using the same pattern that we used for accuracy. For example here we use the **f1_score** and **brier_score** functions from **sklearn** to cauiclate different perfomrance measures.

In [ ]:
f1_score = sklearn.metrics.f1_score(y_valid, y_pred) 
print("F1 Score: " +  str(f1_score))

In [ ]:
brier_score = sklearn.metrics.brier_score_loss(y_valid, y_pred) 
print("Brier Score: " +  str(brier_score))

**ROC curves** are a useful tool for evalauting the performance of machine learning models trained for binary classification problems (although with the simple decision tree in this exmaple they are not terribly useful). To generate an ROC curve we first have to generate **prediction score** output fromt he trasined model rahter than actual class labels. In **sklearn** we do this using the **predict_proba** method. 

In [ ]:
y_pred_score = my_tree.predict_proba(X_valid)
display(y_pred_score)

The **roc_auc_score** method from **sklearn** will genrate the infrmation needed to generate an ROC cource. It is called in the same way as accuracy - by passing a list of ground truth target labels and a set of predictions. 

In [ ]:
print(sklearn.metrics.roc_auc_score(y_valid, y_pred_score[:, 1]))

Note that **y_pred_score** contains two columns (one for each class) but we only need one of these for generating prediction score-based performance measures. To extract this we use `y_pred_score[:, 1]`. 

To extract the information required to draw an ROC curve **sklearn** provides the **roc_curve** merthod. The returns the false positive rate and true positive rates for a set of thresholds (which are also returned). 

In [ ]:
fpr, tpr, thresh = sklearn.metrics.roc_curve(y_valid, y_pred_score[:, 1])

Using the false positive rate and true positive rate we can caluclate the ROC index as well as drawing the actual **ROC curve**. 

In [ ]:
roc_auc = sklearn.metrics.auc(fpr, tpr)
print(roc_auc)
plt.plot(fpr, tpr)

### Evaluating Using Cross Validation

An alternative to using a single trianing and validation set partition to evaluate a machine learning model is to use  ***k*-fold cross valdiation**. We do this in **sklearn** using the **cross_val_score** method. The key parameters for the **cross_val_score** method are:

- **estimator**: The model object to fit to the data.
- **X**: The descriptive feature  data to fit to.
- **y**: the target feature values to fit to.
- **cv** = 5: The number of folds to perform.
- **scoring** = None: The scoring method to use to assess models.
- **n_jobs** = 1: Number of jobs to run in parallel. -1 uses all available. 
- **verbose**=0: Controls how much output will be produced when methods are called - can be 0 (no output), 1, or 2 (maximum output). 

In [ ]:
scores = sklearn.model_selection.cross_val_score(
                                    sklearn.tree.DecisionTreeClassifier(
                                             criterion = 'entropy', max_depth = 5), 
                                    X_train_plus_valid, y_train_plus_valid, 
                                    cv=10)

The function returns the cross valdiation scores, which we typically aggregate using an average.

In [ ]:
print(scores)
print('Mean: {} Std. dev.: {}'.format(np.mean(scores), np.std(scores)))

We can change the perfomrance measure used using the **scoring** parameter. there is a list of those that can be used here: https://scikit-learn.org/stable/modules/model_evaluation.html In this exmaple we swap to using AUC.

In [ ]:
scores = sklearn.model_selection.cross_val_score(
                                    sklearn.tree.DecisionTreeClassifier(
                                             criterion = 'entropy', max_depth = 5), 
                                    X_train_plus_valid, y_train_plus_valid, 
                                    scoring = 'roc_auc',
                                    cv=10)

In [ ]:
print(scores)
print('Mean: {} Std. dev.: {}'.format(np.mean(scores), np.std(scores)))

### Choosing Hyper-parameters Using a Grid Search

A grid search is a useful way to set hyper-parameter values for machine learning models. The **sklearn** **GridSearchCV** object implements this. To perform a grid search we first set up a parameter search grid as a dictionary where each key is a parameter name and each value is a list of options to be searched. 

In [ ]:
param_grid = {'criterion': ['gini', "entropy"], 
              'splitter': ['best', 'random'],
              'max_depth': list(range(3, 20, 3))}

To perform the actual grid search we follow the usual **sklearn** pattern of creating an object and then fitting it to a dataset using the **fit** function. The key parmaeters when creating a **GridSearchCV** object are:

- **estimator**: The model object to fit to the data.
- **param_grid**: The parameter grid to search. 
- **scoring** = None: The scoring method to use to assess models.
- **n_jobs** = 1: Number of jobs to run in parallel. -1 uses all available. 
- **cv** = 5: The number of folds to perform.
- **refit** = True: Refit an estimator using the best found parameters on the whole dataset.
- **verbose**=0: Controls how much output will be produced when methods are called - can be 0 (no output), 1, or 2 (maximum output). 

To the **fit** function we pass the dataset to use (in this case the combined training and validation partitions). 

In [ ]:
my_tuned_tree = \
    sklearn.model_selection.GridSearchCV( \
            sklearn.tree.DecisionTreeClassifier(min_samples_split=50),
            param_grid, 
            cv=10, 
            scoring = 'roc_auc', 
            verbose = 2, 
            n_jobs=1)
my_tuned_tree.fit(X_train_plus_valid, y_train_plus_valid)

The grid search returns a data structure containing the following attributes:
    
- **cv_results_**: A dictionary of the performance results for the grid search. 
- **best_estimator_**: The model chosen by the saerch. 
- **best_score_**: Mean cross-validated score of the best_estimator.
- **best_params_**: Parameter setting that gave the best results on the hold out data.

In [ ]:
print("Best score: {}".format(my_tuned_tree.best_score_))
print("Best parameters: {}".format(my_tuned_tree.best_params_))

In [ ]:
display(my_tuned_tree.cv_results_)

We can visualise the final tree found by accessing the **best_estimator_** attribute which accesses the trained tree object. 

In [ ]:
viz = dtreeviz.trees.dtreeviz(my_tuned_tree.best_estimator_, 
                              X_train_plus_valid, y_train_plus_valid,
                              target_name="churn",
                              feature_names=X_train.columns)
viz

### Final Evaluation on Test Set

With the final model determined we can now make a set of predictions for the hold-out test set to evaluate the generalisation error of the model. 

In [ ]:
# Make a set of predictions for the test data
y_pred = my_tuned_tree.predict(X_test)

# Print performance details
print(sklearn.metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)